## Run Grid Search on Machine Learning Algorithms

In [21]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import datetime

In [37]:
%run ml_helpers.ipynb

               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
1080332  11567335     False   434  2019     1     2   13    15  Second  0.02   
908920   11371006     False   414  2018     6    22    2     6   First  0.32   
1031434  11497552     False   421  2018    11    44    4    12  Second  0.59   
981640   11431660     False   424  2018     8    35   30    19   Third  0.00   
1140544  11658653      True   124  2019     4    16   16    12  Second  0.00   
893810   11316372     False   532  2018     5    20   14    17   Third  1.48   
807857   11198797      True  1123  2018     1     2    8    20   Third  0.00   
884319   11304770     False   512  2018     5    18    2    11  Second  0.22   
859973   11271200     False   834  2018     3    13   29     4   First  0.07   
997510   11505905     False  1612  2018     9    38   20     9  Second  0.00   
994139   11448043     False  2424  2018     9    37   15    22   Third  0.00   
1005064  11462213     False  1013  2018 

Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [3]:
#Import bucketized data

#Run convert_to_categorical
#data = ml_helpers.convert_to_categorical(data, [columns_to_convert])

#Run prep_data(df, y, num_years, year_col, vars_to_onehot):
#data_list = ....

#If bucketized data doesn't have spatial lag, make it have spatial lag

In [26]:
def grid_search_results(model_pipe, model_name, grid_params, scoring, X_train, y_train):
    '''
    Runs gridsearch for data on a given year to find best hyperparameters
    Inputs:
        model_pipe: sklearn Pipeline object with a model and name
        grid_params (dict): maps hyperparameters to potential options
        scoring (list of strings): a list of the ways to score
    Ouputs:
        results (pd.DataFrame): the cross-validated scores for each model
    '''
    gridsearch = GridSearchCV(model_pipe, 
                          grid_params, 
                          scoring = scoring, 
                          refit = scoring[0],    
                          cv=10, 
                          n_jobs=-1)
    fit = gridsearch.fit(X_train, y_train)
    results = pd.DataFrame(fit.cv_results_)
    #results = results[["params", "mean_test_precision"]]
    return results

In [27]:

pipe = Pipeline([('clf' , LogisticRegression(random_state=0))])

X, y = load_iris(return_X_y=True)

y = np.where(y==2, 1, y)
y
clf = LogisticRegression(random_state=0)

params = [
    {
        "clf__C":[.001, .01, .1, 1, 10, 100, 1000],
    }
    ]

grid_search_results(pipe, "clf", params, ["precision", "recall"], X, y)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__C,params,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,...,split3_test_recall,split4_test_recall,split5_test_recall,split6_test_recall,split7_test_recall,split8_test_recall,split9_test_recall,mean_test_recall,std_test_recall,rank_test_recall
0,0.008289,0.001683,0.003879,0.002260,0.001,{'clf__C': 0.001},0.666667,0.666667,0.666667,0.666667,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.009731,0.001317,0.002854,0.000592,0.01,{'clf__C': 0.01},1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.014817,0.002839,0.002431,0.000050,0.1,{'clf__C': 0.1},1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.012444,0.000903,0.003178,0.002496,1,{'clf__C': 1},1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.013982,0.002600,0.002345,0.000184,10,{'clf__C': 10},1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
5,0.012339,0.000648,0.002681,0.000611,100,{'clf__C': 100},1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
6,0.015904,0.002634,0.002391,0.000646,1000,{'clf__C': 1000},1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


In [76]:
def average_grid_searches(model, model_name, 
                          grid_params, scoring, 
                          data_list, y, test_year):
    '''
    Model: sklearn model object
    model_name (string): a name for the model
    grid_params (dict): maps hyperparameters to potential options
    scoring (list of strings): a list of the ways to score
    data (pd.DataFrame): a pandas dataframe with all the data
    test_year (int): The year we seek to predict
    
    Idea to concatenate rows of dfs came via the first answer on from this stack exchange: 
    https://stackoverflow.com/questions/44515888/compute-average-mean-across-dataframes-in-python-pandas
    
    Get mean of col rows help from here: 
    https://stackoverflow.com/questions/33750326/compute-row-average-in-pandas
    '''
    print("test year is:", test_year)
    target_years_in_train = list(np.arange(2015,test_year))
    target_years_in_train.pop(0)
    print(target_years_in_train)
    results_dfs = []
    
    for data_set in data_list:
        train_df = data_set[0]
        test_df = data_set[1]
        year = data_set[2]
        X_train, y_train = split_X_y(train_df, y)
        X_test, y_test = split_X_y(test_df, y)
        
        model_pipe = Pipeline([(model_name, model)])
        
        #model_pipe, model_name, grid_params, scoring, X_train, y_train
        results_df = grid_search_results(model_pipe, model_name, 
                                         grid_params, scoring, 
                                         X_train, y_train)
        results_df = results_df[["mean_test_precision", "params"]]
        results_df = results_df.add_suffix("_" + str(year))
        results_dfs.append(results_df)
    
    all_results = pd.concat(results_dfs, axis = 1)
    all_results["mean"] = all_results.mean(axis=1, numeric_only=True)
    all_results = all_results[["params_" + str(test_year-1), "mean"]]
    all_results.sort_values("mean", inplace=True, ascending=False)
    #results_to_return = all_results["params", 'mean'].sort_values("mean")
    #return results_to_return

    log_reg_results["mean"]=log_reg_results.mean(axis=1, numeric_only=True)


    
    return all_results
    

### Test the code:

In [69]:
#Get data set up
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv("../intermediate_data/df_2015_to_present.csv")
data["was_arrested"]=data["Arrest"].astype("float")
data = data.drop(["Arrest", "category_1", "category_2", "Domestic", "ID", "Week", "Day", "Hour"], axis = 1)
data = convert_to_categorical(data, ["Beat"])
data_small = data.sample(frac=0.001)

data_small_ready = prep_data(data_small, "was_arrested",
                                        1, "Year", ["Year", "Month", "Beat", "Watch"])

         Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  count_l_stops  \
1187632   513  2019      6   First  0.00   0.0    79    58            0.0   
1146404  1215  2019      4   Third  0.00   0.0    65    45            1.0   
1238651  1011  2019      8   Third  0.42   0.0    78    68            0.0   
1289312   825  2019     10   Third  0.00   0.0    52    41            0.0   
1188520   312  2019      6  Second  0.37   0.0    75    56            2.0   
...       ...   ...    ...     ...   ...   ...   ...   ...            ...   
1313722  2023  2019     12   Third  0.00   0.0    44    32            2.0   
1154945  1821  2019      5  Second  0.04   0.0    72    47            1.0   
1130979  2232  2019      4   Third  0.00   0.0    57    38            0.0   
1246196  1222  2019      8   Third  0.00   0.0    74    62            2.0   
1292768   835  2019     11   First  0.00   0.0    54    31            0.0   

         count_bus_stops  ...  count_restaurants  count_bars  count_daycare

Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [46]:
data_small_ready[0][0].head()

,PRCP,SNOW,TMAX,TMIN,count_l_stops,count_bus_stops,count_metra_stops,count_restaurants,count_bars,count_daycares,...,Beat_2524,Beat_2525,Beat_2531,Beat_2532,Beat_2533,Beat_2534,Beat_2535,Watch_First,Watch_Second,Watch_Third
980474,1.844126,-0.243559,0.739656,1.024898,-0.566004,-1.066694,-0.388701,-0.727663,-0.545863,-0.861290,...,0,0,0,0,0,0,0,0,1,0
919027,-0.380824,-0.243559,1.259845,0.977451,-0.566004,-0.626305,-0.388701,-0.403807,-0.516303,-0.861290,...,0,0,0,0,0,0,0,1,0,0
1065832,-0.380824,-0.243559,-0.907609,-0.778088,-0.566004,-2.199122,-0.388701,-0.915158,-0.545863,-1.214356,...,0,0,0,0,0,0,0,0,1,0
816287,-0.351928,-0.243559,-0.517468,-0.256171,0.512790,-0.437567,1.132005,-0.727663,-0.516303,-0.155159,...,0,0,0,0,0,0,0,0,0,1
872391,-0.294138,1.487555,-1.081006,-0.778088,-0.566004,-0.374655,-0.388701,-0.471987,-0.545863,0.550972,...,0,0,0,0,0,0,0,0,1,0


In [47]:
data_small_ready[0][2]

2019

In [77]:
#get other ML things going:
from sklearn.linear_model import LogisticRegression
#name = "log_reg"

params = [
    {
        "log_reg__C":[.001, .01, .1, 1, 10, 100, 1000],
    }
    ]

log_reg_results = average_grid_searches(
                            LogisticRegression(), "log_reg", 
                          params, ["precision", "recall"], 
                          data_small_ready, "was_arrested", 2020)

log_reg_results["mean"]=log_reg_results.mean(axis=1, numeric_only=True)


test year is: 2020
[2016, 2017, 2018, 2019]


In [78]:
log_reg_results

,params_2019,mean
6,{'log_reg__C': 1000},0.259534
5,{'log_reg__C': 100},0.244997
4,{'log_reg__C': 10},0.241591
3,{'log_reg__C': 1},0.186250
2,{'log_reg__C': 0.1},0.050000
0,{'log_reg__C': 0.001},0.000000
1,{'log_reg__C': 0.01},0.000000
